In [23]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect


In [24]:
 csv_file = "data-csv/goals.csv"
goals = pd.read_csv(csv_file)
goals.head()

,key_id,goal_id,tournament_id,tournament_name,match_id,match_name,match_date,stage_name,group_name,team_id,...,shirt_number,player_team_id,player_team_name,player_team_code,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,penalty
0,1,G-0001,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,19',19,0,first half,0,0
1,2,G-0002,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,40',40,0,first half,0,0
2,3,G-0003,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,43',43,0,first half,0,0
3,4,G-0004,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-28,...,0,T-28,France,FRA,87',87,0,second half,0,0
4,5,G-0005,WC-1930,1930 FIFA World Cup,M-1930-01,France v Mexico,1930-07-13,group stage,Group 1,T-44,...,0,T-44,Mexico,MEX,70',70,0,second half,0,0


In [25]:
new_goals = goals[['goal_id', 'tournament_id','match_id', 'team_id','player_id','minute_label','minute_regulation','minute_stoppage','match_period','own_goal','penalty']].copy()

In [26]:
new_goals['year']= new_goals['tournament_id'].str.strip().str[-4:]
new_goals = new_goals.loc[(new_goals['year'] > '1997')]
new_goals.head()

,goal_id,tournament_id,match_id,team_id,player_id,minute_label,minute_regulation,minute_stoppage,match_period,own_goal,penalty,year
1584,G-1585,WC-1998,M-1998-01,T-09,P-05926,5',5,0,first half,0,0,1998
1585,G-1586,WC-1998,M-1998-01,T-61,P-05404,38',38,0,first half,0,1,1998
1586,G-1587,WC-1998,M-1998-01,T-09,P-04230,74',74,0,second half,1,0,1998
1587,G-1588,WC-1998,M-1998-02,T-45,P-02981,37',37,0,first half,0,0,1998
1588,G-1589,WC-1998,M-1998-02,T-45,P-02819,60',60,0,second half,0,0,1998


In [ ]:
#Connect to local database

protocol = 'postgresql'
username = '<user name>'
password = '<password>'
host = 'localhost'
port = 5432
database_name = '#databasename#'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

In [ ]:
insp.get_table_names() 

In [ ]:
# Use pandas to load csv converted DataFrame into database

new_customer_data_df.to_sql(name='goals', con=engine, if_exists='append', index=False) 